# How to find similar documents based on the similarity matrix?



# Read news dataset

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('vaccine_preprocessed_2.csv', sep='|')

In [3]:
df.head(1)

,item_id,categories,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
0,1,AZ,澎湖確診數升 兒童疫苗接種率已逾7成,澎湖COVID-19（2019冠狀病毒疾病）確診數攀升，澎湖縣政府今天公布新增182例，還有...,0.0,"['澎湖5至11歲兒童疫苗接種種率已逾7成', '澎湖縣政府今天公布新增182例確診案例',...","[('澎湖', 5), ('疫苗', 5), ('接種', 4), ('確診', 3), (...","['澎湖', 'COVID-19', '（', '2019', '冠狀', '病毒', '疾...","['澎湖', '冠狀', '病毒', '疾病', '確診數', '澎湖縣', '政府', '...","[NerToken(word='澎湖', ner='GPE', idx=(0, 2)), N...","[('澎湖', 'Nc'), ('COVID-19', 'FW'), ('（', 'PARE...",https://news.google.com/articles/CBMiMmh0dHBzO...


# Step 1: Find articles by catagory or user keyword query

In [6]:
#-- Given a category, get the latest news
def get_userkeyword_cate_latest_news(cate, user_keywords):
    # get the last news (the latest news)
    # df_cate = df_cate.tail(10)  # Only 10 pieces
    # only return 10 news
    # proceed filtering: news category
    # and or 條件

    # end date: the date of the latest record of news
    end_date = df.date.max()    
    # start date 昨天新聞過濾
    days = 1     
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(days=days)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)

    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    # category condition
    condition = (df.category == cate) 
    
    # (3) query keywords condition使用者輸入關鍵字條件and
    # 若未輸入關鍵字，結果是true，因為"" in text 不管text字串內容為何，運算結果為True
    condition = condition & df.content.apply(lambda text: all(
            (qk in text) for qk in user_keywords))  # 寫法:all()
    
    
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    print(df_query.shape)

    # 隨機挑選4篇
    if len(df_query) >= 4:
        df_query = df_query.sample(4) # Sample only 4 pieces 若文章數不足會報錯!
    else:
        df_query = df_query.tail(4) # Only latest 4 pieces
    
    items = []
    for i in range( len(df_query)):
        item_id = df_query.iloc[i].item_id    
        title = df_query.iloc[i].title
        content = df_query.iloc[i].content
        category = df_query.iloc[i].category
        link = df_query.iloc[i].link
        photo_link = df_query.iloc[i].photo_link
        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link=""

        news_info = {
            "item_id": item_id, 
            "category": category, 
            "title": title,
            "content": content, 
            "link": link,
            "photo_link": photo_link
        }

        items.append(news_info)
    return items

In [7]:
cate='政治'
user_keywords='烏克蘭'
get_userkeyword_cate_latest_news(cate, user_keywords)

(3, 14)


[{'item_id': 'aipl_20220314_1',
  'category': '政治',
  'title': '外交部援烏物資已募4000箱 吳釗燮感謝捐贈民眾',
  'content': '民眾捐贈烏克蘭的愛心物資持續湧入外交部，截至今天傍晚累計已收到約4000箱，外交部長吳釗燮中午親自到現場為協助整理物資的志工加油，並對捐贈民眾表達感謝。外交部晚間發布新聞稿指出，外交部從7日開始向民間募集捐贈烏克蘭難民的物資，獲得熱烈響應，親赴外交部捐贈物資的民眾約1730人，加上郵寄包裹，目前約已收到4000箱物資，品項以醫療口罩、毛毯、女性衛生用品、尿片、餅乾等為主，募集活動將持續到18日。外交部表示，為了感謝捐贈民眾，與在現場辛苦分類整理的志工、外交部人員，吳釗燮今天中午特別前往外交部西側門地下停車場視察，吳釗燮與在場的慈濟等民間慈善組織志工，以及其他自發到場幫忙的善心人士親切互動，對於也有烏克蘭旅台人士自願擔任義工在現場協助，吳釗燮特別致意慰問。根據外交部提供的照片，到場幫忙的烏克蘭志工是極為關心家鄉情勢的網紅佳娜。外交部再度提醒有意捐贈物資的民眾，捐贈物品請依照外交部網站所公布的清單為限，切勿捐贈或郵寄二手物品或衣物。送到外交部的捐贈物品務必為全新物品、未拆封包裝、有效期至少6個月以上，以免造成整理及後續轉運捐贈的困擾。募集截止時間是3月18日下午5時以前，民眾可以用面送或郵寄清單所列的20類物品及 14 類藥品至外交部。外交部除感謝熱心民眾踴躍捐贈援助烏克蘭人道物資外，也感謝許多志工義務幫忙、貢獻己力。外交部對各界人士奉獻時間與精神投入國際人道援助，表達最高的敬意。',
  'link': 'https://www.cna.com.tw/news/aipl/202203140364.aspx',
  'photo_link': 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20220314/1024x768_20220314000190.jpg'},
 {'item_id': 'aipl_20220314_12',
  'category': '政治',
  'title': '賑濟烏克蘭善款 累計突破6億元',
  'content': '財團法人賑災基金會今天表示，賑濟烏克蘭專戶開設到今天下午4時10

In [8]:
cate='政治'
user_keywords=[] # 未輸入關鍵字
get_userkeyword_cate_latest_news(cate, user_keywords)

(20, 14)


[{'item_id': 'aipl_20220314_15',
  'category': '政治',
  'title': 'NCC前專委喬建中釀玉山森林大火 監院通過彈劾移送懲戒法院',
  'content': '監察院今天說，NCC前專門委員喬建中帶團途經玉山國家公園，任由隊員違法就地砍伐、生火，釀森林大火，有損國家文官官箴及政府形象，監察院全數通過彈劾，移送懲戒法院審理。監察院透過新聞稿表示，國家通訊傳播委員會（NCC）前專門委員喬建中在民國110年5月15、16日帶團途經玉山國家公園杜鵑營地，任由隊員違法就地砍伐、取柴及生火，釀成延燒12天、近80公頃林地的重大森林火災。經監察委員葉宜津、蔡崇義、賴鼎銘調查相關事證後，認為喬建中利用假日從事登山活動，用火不慎，引發森林大火，雖屬非執行職務的違法行為，但身為通傳會法制人員，負責主管法規解釋與疑義並協助訴訟案件等，卻利用其法律智識規避刑責，而隱瞞就地取柴、違法生火等情，還辯稱是因踢倒爐火致災等言論，嚴重敗壞國家文官及政府形象，核有重大違失。經監察院彈劾審查會今天審查，全數通過彈劾，全案移送懲戒法院審理。彈劾理由指出，喬建中是具豐厚法律專業、法制與公關實務經驗的國家高階簡任文官，明知在國家公園內營火屬不法行為，非但不思遵守法律，行前主動提出杜鵑營地、南營地及大水窟等3營地生火可行性，且已知杜鵑營地離水源來回距離1.5公里，卻任由同行隊友攜帶鋸子，在杜鵑營地就地砍伐、取柴及生火，引發林火致災。監察院表示，喬建中在案發當天接受警方詢問時，隱瞞前晚在杜鵑營地生火事實，並辯稱是因5月16日凌晨2時準備早餐時，不慎踢倒爐火導致地火悶燒釀災，意圖規避森林法刑事責任，企圖以其法律智識誤導檢方及視聽大眾。經消防與林務機關鑑定及認定是砍伐樹枝生火致災而戳破謊言，有失國家高階文官品位，已損及政府及機關形象，顯已違反公務員誠實謹慎義務。監察院指出，喬建中在108年6月到110年6月的2年期間，向玉山國家公園管理處提出12次入園申請，其中11次都擔任登山隊領隊；105年7月到110年7月的5年期間，向警政署提出41次入山申請。110年4月間，獲邀隨同行政院政務委員踏勘玉山首登路線及八通關沿線山屋整建規劃案，足以證明喬建中深入山林頻繁且經驗豐富。監察院說，因喬建中任職於通傳會，更應知台灣山區通訊不良一向是山林救災救難的限制，卻背離其職

In [9]:
user_keywords=['烏克蘭','外交部']
df.content.apply(lambda text: all((qk in text) for qk in user_keywords))

0       True
1      False
2      False
3      False
4      False
       ...  
208    False
209    False
210    False
211    False
212    False
Name: content, Length: 213, dtype: bool

In [10]:
user_keywords=[]
df.content.apply(lambda text: all((qk in text) for qk in user_keywords))

0      True
1      True
2      True
3      True
4      True
       ... 
208    True
209    True
210    True
211    True
212    True
Name: content, Length: 213, dtype: bool

In [11]:
text='This is a sentence.'
"" in text

True

# Step 2: Get news content for displaying on django website

In [12]:
itemid = 'aipl_20220314_16'

In [13]:
df.loc[df.item_id == itemid]

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
15,aipl_20220314_16,2022-03-14,政治,商總理事長：企業界盼5月開放國門、入境免隔離,全國商業總會理事長許舒博今天出席一場記者會時透露，工商團體日前拜會蔡總統，希望在疫情緩和下，...,0.0,"['被問到全國工業總會爆發理事長王文淵與秘書長不合的茶壺風暴', '跳電不是用電不足的問題'...","[('許舒博', 7), ('台灣', 6), ('問題', 5), ('疫情', 4), ...","['全', '國', '商業', '總會', '理事長', '許舒博', '今天', '出席...","['商業', '總會', '理事長', '許舒博', '出席', '記者會', '工商', ...","[NerToken(word='許舒博', ner='PERSON', idx=(9, 12...","[('全', 'Neqa'), ('國', 'Nc'), ('商業', 'Na'), ('總...",https://www.cna.com.tw/news/aipl/202203140210....,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...


## Define function: get_news_content()

In [14]:
# -- Given a item_id, get document information
def get_news_content(item_id):
    df_item = df[df.item_id == item_id]
    title = df_item.iloc[0].title   
    content = df_item.iloc[0].content
    category = df_item.iloc[0].category
    link = df_item.iloc[0].link
    date = df_item.iloc[0].date
    photo_link = df_item.iloc[0].photo_link
    # if photo_link value is NaN, replace it with empty string 
    if pd.isna(photo_link):
        photo_link=''

    news_info = {
        "item_id": item_id,
        "category": category,
        "title": title,
        "content": content,
        "link": link,
        "date": date,
        "photo_link": photo_link
    }

    return news_info

In [15]:
get_news_content(itemid)

{'item_id': 'aipl_20220314_16',
 'category': '政治',
 'title': '商總理事長：企業界盼5月開放國門、入境免隔離',
 'content': '全國商業總會理事長許舒博今天出席一場記者會時透露，工商團體日前拜會蔡總統，希望在疫情緩和下，5月可以全面開放國門、返國免隔離；蔡總統對此相當重視，當場指示行政院副院長沈榮津將意見帶回去與院長蘇貞昌研議。被問到全國工業總會爆發理事長王文淵與秘書長不合的茶壺風暴，許舒博說，他對工總內部情形不瞭解，但商總體系健全，絕不會發生秘書長未經他同意，就把公文發出去的事；他也對追問的媒體打趣說，「大家放心，我不會請辭」。全國商業總會今天舉辦「第四屆品牌金箔獎」海外僑台商組徵件記者會。對於媒體詢問有關停電懲處事件看法，許舒博表示，他過去擔任立委，很瞭解台電，跳電不是用電不足的問題，但台灣電網很脆弱也是問題，政府應思考如何永備發電量，否則將來一樣會面對缺電、跳電問題。許舒博說，大家都不喜歡核電核能，但這次核三廠的備載電源還是掛上去，因為沒有其他可以掛；台灣遇到乾旱，水力發電就掛不下去，台灣河流短小湍急，應思考怎樣留住夏季雨量運用，不要讓雨水一下就流到海洋去。許舒博強調，工商企業界對用電較敏感，反映問題並無攻擊政府的意思，國家是一體的，大家應該一起承擔、一起走出去，不是誰說的就一定對；跳電不管是微幅或大面積都眾所不樂見，不能成為常態，有人為疏失該檢討，有機器設備問題就要汰換。許舒博表示，工商界日前跟蔡總統報告時，很多人都感嘆現在是「出得去、回不來」，因為外國不用隔離，回來關十天對企業就損失很大。許舒博說，疫情期間台灣製造業很好，由於台灣沒疫情、上工率高，外國大量請台灣工廠代工，讓製造業突飛猛進；現在各國都開放了，製造業就沒那麼好過，工商界希望政府能跟著國際腳步，調整疫情管制措施。',
 'link': 'https://www.cna.com.tw/news/aipl/202203140210.aspx',
 'date': '2022-03-14',
 'photo_link': 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20220314/1024x768_20220314000113.jpg'}

# Step 3: Given an item_id, find topk similar articles
    Input: item_id (one of item_ids in the word2vec model)

    Output: top n documents ranked by similarity

    Using model.docvecs.most_similar([item_id])

    (在訓練資料集內的)某篇文件與 哪幾篇最相似？相似度多少？

## Prepare article index

In [16]:
from operator import itemgetter
import pandas as pd
import numpy as np

In [17]:
news_sim_martrix = np.load('news_sim_martrix.npy')

In [18]:
item_id2idx={}
idx2item_id={}
for id, i in df.item_id.items():
    item_id2idx[i] = id
    idx2item_id[id] = i

## Find topk similar articles

In [19]:
item_id = 'acn_20220313_20'
topk=3
sim_dict = {}
idx = item_id2idx[item_id]
for i, value in enumerate(news_sim_martrix[idx]):
    sim_dict[i]=value
similar_items = sorted(sim_dict.items(), key= itemgetter(1), reverse=True)[1:topk+1] # topk+1 多取一筆 有包含本身這一筆


In [20]:
similar_items

[(196, 0.97362715), (146, 0.9724167), (200, 0.9684948)]

## Get article information to display

In [21]:
items = []
for idx, score in similar_items:
    item = df.iloc[idx]
    item_id = item.item_id
    title = item.title
    content = item.content
    category = item.category
    link = item.link
    photo_link = item.photo_link
    # if photo_link value is NaN, replace it with empty string 
    if pd.isna(photo_link):
        photo_link=''

    score = round(score, 2)
    news_info = {
        "category": category, 
        "title": title, 
        "link": link,
        "id": item_id, 
        'score': score, 
        "photo_link": photo_link
        }
    items.append(news_info)

In [22]:
items

[{'category': '兩岸',
  'title': '美國立法地圖正確標示台灣  北京：停止掏空一中',
  'link': 'https://www.cna.com.tw/news/acn/202203140294.aspx',
  'id': 'acn_20220314_4',
  'score': 0.97,
  'photo_link': ''},
 {'category': '國際',
  'title': '遭港府要求關站 英人權組織香港觀察誓言不會噤聲',
  'link': 'https://www.cna.com.tw/news/aopl/202203140334.aspx',
  'id': 'aopl_20220314_7',
  'score': 0.97,
  'photo_link': ''},
 {'category': '兩岸',
  'title': '港府效忠學北京官腔官調  凸顯香港自治被剝奪',
  'link': 'https://www.cna.com.tw/news/acn/202203140197.aspx',
  'id': 'acn_20220314_8',
  'score': 0.97,
  'photo_link': 'https://imgcdn.cna.com.tw/www/webphotos/WebCover/420/20220314/3811x2858_389455461856.jpg'}]

## Get topk similar articles based on a certain item_id

In [23]:
#-- Given item_id, get three similar article information
def get_topk_similar_articles(item_id):

    ## Find topk similar articles
    topk=3
    sim_dict = {}
    idx = item_id2idx[item_id]
    for i, value in enumerate(news_sim_martrix[idx]):
        sim_dict[i]=value
    similar_items = sorted(sim_dict.items(), key= itemgetter(1), reverse=True)[1:topk+1] # topk+1 多取一筆 有包含本身這一筆

    ## Get article information to display
    items = []
    for idx, score in similar_items:
        item = df.iloc[idx]
        item_id = item.item_id
        title = item.title
        content = item.content
        category = item.category
        link = item.link
        photo_link = item.photo_link
        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link=''

        score = round(score, 2)
        news_info = {
            "category": category, 
            "title": title, 
            "link": link,
            "id": item_id, 
            'score': score, 
            "photo_link": photo_link
            }
        items.append(news_info)

    return items

In [24]:
item_id = 'acn_20220313_20'
get_topk_similar_articles(item_id)

[{'category': '兩岸',
  'title': '美國立法地圖正確標示台灣  北京：停止掏空一中',
  'link': 'https://www.cna.com.tw/news/acn/202203140294.aspx',
  'id': 'acn_20220314_4',
  'score': 0.97,
  'photo_link': ''},
 {'category': '國際',
  'title': '遭港府要求關站 英人權組織香港觀察誓言不會噤聲',
  'link': 'https://www.cna.com.tw/news/aopl/202203140334.aspx',
  'id': 'aopl_20220314_7',
  'score': 0.97,
  'photo_link': ''},
 {'category': '兩岸',
  'title': '港府效忠學北京官腔官調  凸顯香港自治被剝奪',
  'link': 'https://www.cna.com.tw/news/acn/202203140197.aspx',
  'id': 'acn_20220314_8',
  'score': 0.97,
  'photo_link': 'https://imgcdn.cna.com.tw/www/webphotos/WebCover/420/20220314/3811x2858_389455461856.jpg'}]

# Django views.py

In [25]:
from django.shortcuts import render
from django.http import JsonResponse
import pandas as pd
from django.views.decorators.csrf import csrf_exempt

from operator import itemgetter
import numpy as np
from datetime import datetime, timedelta


# (2) Load news data--approach 2
def load_df_data_v1():
    
    global df # global variable
    df = pd.read_csv('app_news_rcmd_bert/dataset/cna_news_200_preprocessed.csv',sep='|')
    global news_sim_martrix
    news_sim_martrix = np.load('app_news_rcmd_bert/dataset/news_sim_martrix.npy')
    global item_id2idx
    item_id2idx={}

    for id, i in df.item_id.items():
        item_id2idx[i] = id

# call load data function when starting server
load_df_data_v1()


#-- home page
def home(request):
    return render(request, "app_news_rcmd_bert/home.html")

#-- API: input category
@csrf_exempt
def api_query_keyword_cate_news(request):
    cate = request.POST['category']
    user_keywords = request.POST['input_keywords']
    user_keywords = user_keywords.split()
    response = get_userkeyword_cate_latest_news(cate, user_keywords)
    return JsonResponse({"latest_news": response})

#-- API: input news_id, and then get news information
@csrf_exempt
def api_news_content(request):
    item_id = request.POST['item_id']
    content = get_news_content(item_id)
    related = get_topk_similar_articles(item_id)
    # print(related)
    return JsonResponse({"news_content": content, "related_news": related})


#-- Given a category, get the latest news
def get_userkeyword_cate_latest_news(cate, user_keywords):
    # get the last news (the latest news)
    # df_cate = df_cate.tail(10)  # Only 10 pieces
    # only return 10 news
    # proceed filtering: news category
    # and or 條件

    # end date: the date of the latest record of news
    end_date = df.date.max()    
    # start date 昨天新聞過濾
    days = 1     
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(days=days)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)

    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    # category condition
    condition = (df.category == cate) 
    
    # (3) query keywords condition使用者輸入關鍵字條件and
    # 若未輸入關鍵字，結果是true，因為"" in text 不管text字串內容為何，運算結果為True
    condition = condition & df.content.apply(lambda text: all(
            (qk in text) for qk in user_keywords))  # 寫法:all()
    
    
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    # 隨機挑選4篇
    if len(df_query) >= 4:
        df_query = df_query.sample(4) # Sample only 4 pieces 若文章數不足會報錯!
    else:
        df_query = df_query.tail(4) # Only latest 4 pieces
    
    items = []
    for i in range( len(df_query)):
        item_id = df_query.iloc[i].item_id    
        title = df_query.iloc[i].title
        content = df_query.iloc[i].content
        category = df_query.iloc[i].category
        link = df_query.iloc[i].link
        photo_link = df_query.iloc[i].photo_link
        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link=""

        news_info = {
            "item_id": item_id, 
            "category": category, 
            "title": title,
            "content": content, 
            "link": link,
            "photo_link": photo_link
        }

        items.append(news_info)
    return items

# -- Given a item_id, get document information
def get_news_content(item_id):
    df_item = df[df.item_id == item_id]
    title = df_item.iloc[0].title   
    content = df_item.iloc[0].content
    category = df_item.iloc[0].category
    link = df_item.iloc[0].link
    date = df_item.iloc[0].date
    photo_link = df_item.iloc[0].photo_link
    # if photo_link value is NaN, replace it with empty string 
    if pd.isna(photo_link):
        photo_link=''

    news_info = {
        "item_id": item_id,
        "category": category,
        "title": title,
        "content": content,
        "link": link,
        "date": date,
        "photo_link": photo_link
    }

    return news_info

#-- Given item_id, get three similar article information
def get_topk_similar_articles(item_id):

    ## Find topk similar articles
    topk=2
    sim_dict = {}
    idx = item_id2idx[item_id]
    for i, value in enumerate(news_sim_martrix[idx]):
        sim_dict[i]=value
    similar_items = sorted(sim_dict.items(), key= itemgetter(1), reverse=True)[1:topk+1] # topk+1 多取一筆 有包含本身這一筆

    ## Get article information to display
    items = []
    for idx, score in similar_items:
        item = df.iloc[idx]
        item_id = item.item_id
        title = item.title
        content = item.content
        category = item.category
        link = item.link
        photo_link = item.photo_link
        # if photo_link value is NaN, replace it with empty string 
        if pd.isna(photo_link):
            photo_link=''

        score = round(float(score), 2)
        news_info = {
            "category": category, 
            "title": title, 
            "link": link,
            "id": item_id, 
            'score': score, 
            "photo_link": photo_link
            }
        items.append(news_info)

    return items

print("app Bert based news recommendation was loaded!")


FileNotFoundError: [Errno 2] No such file or directory: 'app_news_rcmd_bert/dataset/cna_news_200_preprocessed.csv'